In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from peakfinder import find_peaks

In [2]:
# http://timeseriesclassification.com/description.php?Dataset=ECG200

In [3]:
df = pd.read_csv("./data/ECG/ECG/ECG_TRAIN.txt", sep=",", header=None)

FileNotFoundError: [Errno 2] File b'./data/ECG/ECG/ECG_TRAIN.txt' does not exist: b'./data/ECG/ECG/ECG_TRAIN.txt'

In [ ]:
df.head()

In [ ]:
df.loc[0,1:].plot()
plt.title("Normal")
plt.show()

In [ ]:
df.loc[1,1:].plot()
plt.title("Not Normal")
plt.show()

### How do they differ on their FF?

In [ ]:
normal = df.loc[0,1:].values
event =  df.loc[1,1:].values

In [ ]:
from scipy.fftpack import fft

In [ ]:
fft_vals = fft(normal)
f_vals = range(len(normal))

plt.plot(f_vals, fft_vals)
plt.title("Normal")

In [ ]:
fft_vals = fft(event)
f_vals = range(len(event))

plt.plot(f_vals, fft_vals)
plt.title("Event")

## How do they differ in ther PSD?

Closely related to the Fourier Transform is the concept of Power Spectral Density.

Similar to the FFT, it describes the frequency spectrum of a signal. But in addition to the FFT it also takes the power distribution at each frequency (bin) into account. Generally speaking the locations of the peaks in the frequency spectrum will be the same as in the FFT-case, but the height and width of the peaks will differ. 

Calculation of the Power Spectral density is a bit easier, since SciPy contain a function which not only return a vector of amplitudes, but also a vector containing the tick-values of the frequency-axis.

In [ ]:
from scipy.signal import welch
f_vals, psd_vals = welch(normal)

plt.plot(f_vals, psd_vals)
plt.title("Normal")

In [ ]:
from scipy.signal import welch
f_vals, psd_vals = welch(event)

plt.plot(f_vals, psd_vals)
plt.title("Event")

## Savitzky-Golay

Another idea is to fit the signal locally by smooth polynomials. In this case, we can transform the signal to their approximation and hope to see some differences.

In [ ]:
from scipy.signal import savgol_filter

In [ ]:
plt.plot(savgol_filter(normal,5,2)) #window length of 5, second order polynomials

In [ ]:
plt.plot(savgol_filter(event,5,2))

### Find peaks

In [ ]:
from scipy.signal import find_peaks_cwt

In [ ]:
peaks_normal_idxs = find_peaks_cwt(normal, np.arange(1,len(normal)))

In [ ]:
peaks_event_idxs = find_peaks_cwt(event, np.arange(1,len(event)))

In [ ]:
plt.plot(np.arange(0, len(normal)), normal, '-b+', mec = 'r', markevery= list(peaks_normal_idxs))

In [ ]:
plt.plot(np.arange(0, len(event)), event, '-b+', mec='r', markevery= list(peaks_normal_idxs))

## Using the `find_peaks` function (included)

In [ ]:
# Credit to Marcos Duarte: http://nbviewer.jupyter.org/github/demotu/BMC/blob/master/notebooks/DetectPeaks.ipynb
peaks_normal = find_peaks(normal, show=True, 
                          mpd = 5, threshold=0.1, mph=-1)

In [ ]:
peaks_event = find_peaks(event, show=True, edge ='rising', kpsh=False, mpd=10, mph = -1)

In [ ]:
features = list(peaks_normal) + list(peaks_event)

In [ ]:
X = df.iloc[0:,features]

In [ ]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters=2, random_state=42, n_init=10)

km.fit(X)

In [ ]:
y_preds = km.predict(X)

In [ ]:
y = df.iloc[0:,0]

In [ ]:
y_preds

In [ ]:
from sklearn.metrics import adjusted_rand_score

adjusted_rand_score(y, y_preds)

<div class="alert alert-success">
    <h2>Exercise</h2>:
     <ul>
      <li>Can you improve the clustering results using FFT? How about PSD?</li>
      <li>What happens if you change the clustering algorithms and try to solve the problem as a classification problem?</li>
    </ul>
</div>
